## Basics of Mobile Robotics Final Project
**Basics of Mobile Robotics (MICRO-452)**

**Group 35 :**

CARRETTI Julie\
DOANE Ashton\
BOUGUECHA Meriam\
SCHLUCHTER Julien

## Table of Contents 
- [Introduction - Environment Desciption](#introduction)
- [Computer Vision](#Computer-Vision)
    - [1. Installs for all required elements](#1.-Installs-for-all-required-elements)
    - [2. Import and Setup](#2.-Import-and-Setup)
    - [3. Thymio Vision Class Methods](#3.-Thymio-Vision-Class-Methods)
    - [4. Example usage of this class](#4.-Example-usage-of-this-class)
- [Global Navigation](#Global-Navigation)
- [Local Navigation](#Local-Navigation)
- [Filtering](#Filtering)
- [Motion Control](#Motion-Control)
- [Main](#Main)
- [Sources](#Sources)





## Introduction - Environment Desciption

A 2D map was created on an A0-sized white sheet, featuring printed shapes to represent global obstacles and 3D boxes for local obstacles. The map is discretized into 1920 by 1080 pixels, corresponding to the camera’s resolution. The goal is indicated by a green circle, while the Thymio robot is identified through a blue dot and its front yellow lights, both detected by the camera.


## Computer Vision

This section shows the steps that have gone into using computer vision for this project. We namely implemented 4 features:

1. <b>Image capture </b> - Processing images using open-cv and AUKEY camera.
2. <b>Image Filtering</b> - Smoothing out images using filtering methods such as median or gaussian filters.
3. <b>Edge Detection and Processing </b> - Using canny edge detection and radon/hough transform to develop shapes on the map.
4. <b>Thymio Pose Detection</b> - Detecting the location of the Thymio using pattern matching.
5. <b>Map Generation</b> - Creating the map that will be used for path planning.

### 1. Installs for all required elements
------

In [ ]:
!python3 -m pip install --upgrade tdmclient
!python3 -m pip install --upgrade opencv-python
!python3 -m pip install --upgrade numpy
!python3 -m pip install --upgrade matplotlib

### 2. Import and Setup
------

The libraries used for this program:

1) <b>OpenCV</b> - This library was used to perform basic image processing tasks, such as image capturing, edge detection, 
2) <b>numpy</b> - This library is essential for manipulating images from OpenCV, as these images are represented as numpy arrays.
3) <b>matplotlib</b> - The library allows plotting within the jupyter ide, which is useful for both testing and displaying results for you.

In [ ]:
import cv2 # used for image processing
import numpy as np # used for array manipulation in conjuction with
import matplotlib.pyplot as plt # used for displaying processing steps for your aid!

### 3. Thymio Vision Class Methods
----------

#### Camera Calibration
In order to properly have a scale of the environment, we choose to control the camera position relative to the environment. The implementation here is very straightfoward. Note that we have mapped the corners to a section of A0 paper, creating a known ratio of 1px = 0.75 mm. This will be used as a known ratio to map the Thymio's pixel position to a position in real space, which can then be used in the <u>filtering module</u> along with the wheel speed.

In [ ]:
def calibrateCameraPos():
        """
        Position the camera such that it aligns with the corners of A0 paper as shown. This
        is purely for user setup, and does not return a value. If dots are aligned with the corners of
        A0 paper, ensures that 1 px = 0.9344 mm
        """
        cv2.namedWindow("Camera Calibration") 
        vc = cv2.VideoCapture(0)
        ret = True
        while True:
            ret, frame = vc.read()
            if not ret:
                break
            
            # add calibration circles to frame
            cv2.circle(frame, (360, 90), 5, (0, 0, 255), 5)
            cv2.circle(frame, (frame.shape[1]-360, 90), 5, (0, 0, 255), 5)
            cv2.circle(frame, (360, frame.shape[0]-90), 5, (0, 0, 255), 5)
            cv2.circle(frame, (frame.shape[1]-360, frame.shape[0]-90), 5, (0, 0, 255), 5)

            cv2.imshow("Camera Calibration", frame)
            key = cv2.waitKey(50)
            if key == ord('c'): # Escape and return image on c
                break
        
        vc.release()
        cv2.destroyAllWindows()

#### Camera Capture
Simply a command to capture a frame on command. This is used to initialize the robot. 

In [ ]:
def captureImageOnCommand(cam_num):
        """
        Provides the user with a camera feed, from which the user may input 'C' to
        capture the image provided. Does not complete without user input.
        @param cv2 BGR image, from which we extract edges.
        @returns cv2 grayscale image with detected edges from input img.
        """
        cv2.namedWindow("Camera View")
        vc = cv2.VideoCapture(cam_num)
        ret = True
        while True:
            ret, frame = vc.read()
            if not ret:
                break
            cv2.imshow("Camera View", frame)
            key = cv2.waitKey(50)
            if key == ord('c'): # Escape and return image on c
                break
        
        vc.release()
        cv2.destroyAllWindows()
        return frame

#### Edge Detection

Edge detection has two main steps - filtering and an edge detection algortithm.

1. <u>Filtering</u>: In this project, the median filter was used with a radius of 31 pixels. The motivation for this is as follows:
-  The median filter takes the median grayscale value of the square of pixels around the target point. 

- Compared to other filters, the median filter has the main advantage here of eliminating ALL noise, as opposed to something like the average filter, which still factors in noise.

- Our setup consists of objects with very severe and consistent (meaning that the) transitions from light to dark. The median filter ensures that any edges in the original image which only span a few pixels will be ignored, ensuring that we do not read edges from other things in the environment (i.e. the thymio is ignored).

2. <u>Edge Detection</u> In this project, the canny edge detection method was used. The motivation for this is as follows:
- We are not constrained much with respect to time for this computation. It is relatively quick, and thus we may use the more complex canny method to get a higher quality edge map.
- Canny connects strong edges through weak edges - essentially it is a more robust way to ensure our obstacles are closed shapes.

In [ ]:
def getEdges(img, filter = 'median', edge_method = 'canny', verbose=False):
        """
        Extract detected edges from a provided image.
        @param img(cv2 BGR_image): Image from which we extract edges.
        @param filter (string): Indication of what type of filter to overlay on the image.
        @param edge_method (string): Indication of what type of edge detection method should be used.
        @param verbose (bool): If true, will display each step of the processing.
        @returns cv2 grayscale image with detected edges from input img.
        """
        # First, convert the input image to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply the selected filter
        if filter == 'median':
            filtered_img =  cv2.medianBlur(gray_img, 31)
        elif filter == 'average':
            pass
        elif filter == 'gaussian':
            pass
        else:
            filtered_img = gray_img

        #Apply the selected edge detection method to the filtered image
        if edge_method == 'canny':
            edges = cv2.Canny(filtered_img, 100,200)
        else:
            pass

        # If verbose selected, Display images
        if verbose:
            # Set up plot size
            plt.rcParams["figure.figsize"] = (20,5)
            plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
            
            #Grayscaled Image:
            plt.subplot(1, 3, 1)
            plt.title("Grayscale")
            plt.imshow(gray_img, cmap='gray')

            #Filtered Image:
            plt.subplot(1, 3, 2)
            plt.title("Filtered: " + filter)
            plt.imshow(filtered_img, cmap='gray')

            #Edges + expansion radius Image:
            plt.subplot(1, 3, 3)
            plt.title("Edges: " + edge_method)
            plt.imshow(edges, cmap='gray')

            plt.show()
            
        return edges

#### Pose Detection
Pose detection requires both location of the (x,y) position of the robot, as well as a heading angle which we call theta. 

1. <u>Template Matching for (x,y) Positioning</u>: 

    In order to determine the position of the Thymio, a blue dot was attached above the hole in the Thymio. A picture of this can be found under Templates/blueDot.png. We used this picture as a template for the template matching algorithm provided by OpenCV. This function essentially tests the similarilty of every MxN grid within an image to the template image provided, returning the maximally matching location.

    As we used a colored dot, we chose to use the <b>TM_CCORR_NORMED</b> method. As we are using color as the main distinguisher locating the Thymio, it is important to have the blue color match strongly influence the outcome. For other methods such as TM_SQDIFF_NORMED or TM_CCOEFF, each innacurate color channel can create a large difference, with the accuracy of the blue channel being ignored. Thus, calculating instead a correlation value ensures that highly matching colors are rewarded strongly. This held true under testing, with TM_CCORR_NORMED giving the most accurate results 

    Two major drawbacks of this method are <b> rotation invariance </b> and <b> scale invariance </b>, meaning that the matching algorithm does not respond well to rotations or scaling of the template image. 

    To overcome these drawbacks, two choices were made. First, we chose to make the template a isotropically colored circle. This almost completely removes the problem of rotation invariance, as the template looks the same from all angles. However, note that the template is still required to be a square, and thus the part of the template not covered by the circle is still rotation invariant, but using a full white background seemed to avoid this problem.

2. <u>Color BitMasking for Heading Angle</u>: While determining the angle could be done many ways, we opted to use the front LED on the Thymio in order to avoid any unecessary add-ons to the Thymio. As the environment is very color controlled, there should be no other orange lighting in the frame with high intensity. Thus, we used a bitmask using HSV color scheme. This allowed us to accurately select for only orange colors, with high saturation and high intensity.

3. <u>Thymio Pose Detection</u>: This simply combines the prior two functions, and ensures that the results are reasonable (i.e. the center and front are not more than a thymio radius apart). Returns (x ,y, theta) prediction for the Thymio state.

4. <u>Conversions from pixels to real space</u>: From before, we said we know the ratio is 1 px = 0.75mm within a bounding box we place on the screen. Thus, we simply transform from pixels to real space by adding an offset, and multiplying by 0.75.

In [ ]:
def detectBlueDot(frame, divisions=3, method = 'TM_CCORR_NORMED', templatePath = "Templates/blueDot2.png", verbose=False):
    """
    Note: Does NOT support TM_SQDIFF or SQDIFF_NORMED
    """
    template = cv2.imread(templatePath) #read template as bgr image
    globalMax = 0
    best_approx = ([], 0, 0, 0) #pos/w/h/scale
    
    # resize the template image to a variety of scales, perform matching 
    for scale in np.linspace(0.5, 2.0, divisions)[::-1]:
        resized = cv2.resize(frame, (0,0), fx=scale, fy=scale) #resize copy
        
        # get effective size of rectangle bounding box we are searching
        w, h, c = template.shape
        w = int(w/scale)
        h = int(h/scale)

        meth = getattr(cv2, method)

        # Apply template Matching
        res = cv2.matchTemplate(resized,template,meth)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
        if  max_val > globalMax:
            globalMax = max_val
            best_approx = ([int(max_loc[0]/scale), int(max_loc[1]/scale)], w, h, scale)


    top_left,w,h,scale = best_approx
    bottom_right = (top_left[0] + w, top_left[1] + h)

    if verbose:
        copy = frame.copy()
        cv2.rectangle(copy, top_left, bottom_right, (255, 50, 255), 5)
        plt.subplot(122),plt.imshow(copy,cmap = 'gray')
        plt.show()
    
    x = top_left[0] + int(w/2)
    y = top_left[1] + int(h/2)
    return (x,y) #return center of box

def detectOrangeHeading(frame, reduction = 0.1, THRESHOLD = 50, verbose = False):
    lower_quality = cv2.resize(frame, (0,0), fx = reduction, fy = reduction) # rescale for faster processing
    # Create a mask that looks for only the light indicator for position
    hsv = cv2.cvtColor(lower_quality, cv2.COLOR_BGR2HSV) #convert to hsv for masking
    lower_orange = np.array([0, 0, 230]) # hue/saturation/brightness
    upper_orange = np.array([180, 255, 255]) 
    mask = cv2.inRange(hsv, lower_orange, upper_orange)
    result = cv2.bitwise_and(lower_quality, lower_quality, mask=mask) # image correcting

    centerX = 0
    centerY = 0
    numDataPoints = 0
    for i, row in enumerate(result):
        for j, pixel in enumerate(row):
            if pixel.any() != 0:
                centerX += j
                centerY += i
                numDataPoints += 1
    if numDataPoints < THRESHOLD:
        if verbose: 
            print("Orange not found")
        return (None, None)
    centerX = int(centerX/numDataPoints/reduction) # find average and rescale to full value
    centerY = int(centerY/numDataPoints/reduction)
    
    # If verbose, display each step of processing
    if verbose:
        plt.subplot(121), plt.imshow(result)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.subplot(122), plt.imshow(rgb)
        plt.plot([centerX], [centerY], 'o')
        plt.show()
    pass

    return (centerX, centerY)
        
def getThymioPose(frame, verbose=False):
        """
        Extracts the Thymio pose from a camera feed and returns as a triple of (x,y,theta), relative to the top-left corner of the camera.
        @param frame (np.array): BGR cv2 image to extract position from.
        @returns (x, y, theta, size)
        """
        relevantFrame = frame[90:-90, 360:-360]
        blueX, blueY = ThymioVision.detectBlueDot(relevantFrame, minScale=0.5, maxScale=0.5, divisions=1)
        orangeX, orangeY = ThymioVision.detectOrangeHeading(relevantFrame, reduction=0.3, THRESHOLD=15, filter_size=10)

        if blueX is None or orangeX is None:
            print('Thymio not found.')
            return (None, None, None)
        
        blueX += 360
        blueY += 90
        orangeX += 360
        orangeY += 90

        dx = float(orangeX-blueX)
        dy = -float(orangeY-blueY)
        theta = np.arctan2(dy,dx)

        if np.sqrt(dx**2 + dy**2 ) > 300:
            print('Thymio not found. Distance too large')
            return (None, None, None)

        if verbose:
            plt.rcParams["figure.figsize"] = (16,4)
            plt.imshow(frame)
            plt.quiver(blueX, blueY, dx, dy, color='red')
            plt.show()
        return (blueX, blueY, theta)


def pixelToRealSpace(position):
        """
        Converts a pixel location to a dimension in real space. Coordinate frame centered on the top left corner of the paper.
        As the setup always ensures alignment of the camera to the corners of A0 paper, the ratio is set.
        @param position (x,y): Pixel location on the camera image.
        @returns (x,y) tuple of location in real space in cm.
        """
        # Camera shape (1080, 1920, 3)
        # Paper dimensions (841 x 1189mm)
        # Alignment from calibration such that 1 px = 0.9344 mm
        # return ((position[0]-360)*0.9344/10, (position[1]-90)*0.9344/10)
        return ((position[0]-360)*0.416/10,(position[1]-90)*0.416/10) #TEMPORARY SETUP NOV29

#### Goal Detection
Refer to the above template matching scheme for reasoning of implementation.

In [ ]:
def detectGoal(frame, divisions=4, method = 'TM_CCORR_NORMED', templatePath = "Templates/greenDot2.png", verbose=False):
        """
        Note: Does NOT support TM_SQDIFF or SQDIFF_NORMED
        """
        template = cv2.imread(templatePath) #read template as bgr image
        globalMax = 0
        best_approx = ([], 0, 0, 0) #pos/w/h/scale
        
        # resize the template image to a variety of scales, perform matching 
        for scale in np.linspace(0.5, 2, divisions)[::-1]:
            resized = cv2.resize(frame, (0,0), fx=scale, fy=scale) #resize copy
            
            # get effective size of rectangle bounding box we are searching
            w, h, c = template.shape
            w = int(w/scale)
            h = int(h/scale)

            meth = getattr(cv2, method)

            # Apply template Matching
            res = cv2.matchTemplate(resized,template,meth)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        
            if  max_val > globalMax:
                globalMax = max_val
                best_approx = ([int(max_loc[0]/scale), int(max_loc[1]/scale)], w, h, scale)


        top_left,w,h,scale = best_approx
        bottom_right = (top_left[0] + w, top_left[1] + h)

        if verbose:
            copy = frame.copy()
            plt.rcParams["figure.figsize"] = (16,4)
            cv2.rectangle(copy, top_left, bottom_right, (255, 50, 255), 5)
            plt.imshow(copy,cmap = 'gray')
            plt.show()
        
        x = top_left[0] + int(w/2)
        y = top_left[1] + int(h/2)
        return (x,y) #return center of box

#### Map Generation

Map generation simply used the previously known data in order to generate a map that could be parsed by the path planning module. This simply requires running edge detection on a frame, and then expanding each edge with a radius equal to the Thymio radius. This ensures that the path we follow not only avoids edges with the center of the robot, but actually keeps the entire robot safe from obstacles.

In [ ]:
def getMap(frame, verbose=False):
        """
        Determine the map of the layout by considering thymio position, size, detected edges, and goal position. 
        @param frame (np.array): A camera image
        @returns Tuple (map, start, goal) with types (np.array, [x,y], [x,y]) representing the map of edges, start location
        and goal position for the A* algorithm.
        """
        # Get edge list
        edges = ThymioVision.getEdges(frame)

        # Find start and goal position
        startPos = ThymioVision.detectBlueDot(frame)
        if not startPos:
            print("Thymio start position not found")
        tSize =  100 #50 # based on the calibrated camera, the thymio can be approximated by this radius
        goalPos = ThymioVision.detectGoal(frame)
        if not goalPos:
            print("Goal not found")
            return


        # clear out space around goal and start
        cv2.circle(edges, startPos, radius=int(tSize), thickness=-1, color=0)
        cv2.circle(edges, goalPos, radius=int(tSize), thickness=-1, color=0)
        #radius
        final_map = np.zeros(shape=edges.shape)
        for i, row in enumerate(edges):
            for j, pixel in enumerate(row):
                if pixel == 255:
                    cv2.circle(final_map, (j,i), radius=int(tSize), thickness=-1, color=1)

        return (final_map, startPos, goalPos)

### 4. Example usage of this class 

Please refer to the example video linked to see the outputs of these methods.

----------

In [ ]:
from Modules.ThymioVision import ThymioVision
import matplotlib.pyplot as plt

In [ ]:
# First apply the calibration filter in order to align the camera
ThymioVision.calibrateCameraPos(0)

In [ ]:
# Capture a frame of interest.
frame = ThymioVision.captureImageOnCommand(0)

In [ ]:
# Next, procure edges from camera capture 
edges = ThymioVision.getEdges(frame, verbose=True)

In [ ]:
pose = ThymioVision.getThymioPose(frame, verbose = True)

In [ ]:
map = ThymioVision.getMap(frame, verbose=True)

In [ ]:
# Test color matching from image to locate thymio
ThymioVision.detectOrangeHeading(frame, verbose=True, reduction=0.3, THRESHOLD=1, filter_size=15)

In [ ]:
from Modules.ThymioVision import ThymioVision
# Example usage for live thymio tracking
import cv2
cv2.namedWindow("Camera View")
vc = cv2.VideoCapture(0)
while True:
    ret, frame = vc.read()
    if ret:
        x1, y1, t = ThymioVision.getThymioPose(frame)
        print(ThymioVision.pixelToRealSpace((x1,y1)), t)
        # x1, y1 = detectOrangeHeading(frame, reduction=0.3, THRESHOLD=1, filter_size=10, verbose=False)
        cv2.circle(frame, (x1,y1), 5, (0,0,255), -1)
        cv2.imshow("Camera View", frame)
    key = cv2.waitKey(50)
    if key == ord('c'): # Escape and return image on c
        break

vc.release()
cv2.destroyAllWindows()

## Global Navigation

## Path Planning

For the path planning component, we employed **cell decomposition** combined with the **Best-First Search** strategy, which led us to implement the A* algorithm.

### Disclaimer

The A* pathfinding algorithm was developed by following the pseudocode provided in the lecture *"Space Robotics Systems"*, delivered by Prof. Antonio Genova and Dr. Edoardo Del Vecchio from the Department of Mechanical and Aerospace Engineering, Sapienza University of Rome. During the development process, I used **ChatGPT** by OpenAI and **GitHub Copilot** to help refine the implementation, improve the code structure, and add meaningful comments. These tools provided suggestions that I adapted to meet the specific requirements of this project. This acknowledgment ensures transparency and helps prevent concerns related to plagiarism.


### Implementation Overview
To implement the A* algorithm, we followed this pseudocode:
<div style="text-align: center;">
    <img src="images/pseudocode_Astar.jpg" alt="A* Pseudocode" style="width:40%; height:auto;">
</div>

### Importing Required Libraries

In [ ]:
import math
import numpy as np
import heapq
from PIL import Image, ImageDraw

### NODE Class Overview

The `NODE` class is designed to represent a point in the grid, encapsulating all the necessary attributes and methods for implementing the A* pathfinding algorithm. Below, we describe the key components of the `NODE` class and their roles:

#### **Attributes and Methods**

- `position`: Represents the coordinates of the node on the grid (e.g., `[x, y]`).

- `g`: The **cost** from the starting node to the current node. It is initialized to infinity (`float('inf')`) to indicate that nodes start off as unexplored.

- `h`: The **heuristic estimate** of the cost from the current node to the goal. This value guides the A* algorithm toward the goal.

- `f`: The **total cost**, computed as `f = g + h`. This value is used to prioritize nodes during the search.

- `parent`: A reference to the **parent node** in the path, used for path reconstruction.

- **Comparison (`__lt__`)**:
This method allows nodes to be compared based on their f value. It is crucial because we use heapq, a priority queue that requires comparison for efficient sorting.

- **String Representation (`__repr__`)**:
This method defines how the node should be represented when printed.

- **Path Reconstruction (`reconstruct_path`)**:
This static method is used to reconstruct the optimal path once the goal is reached. Starting from the goal node, it traces back through each node’s parent attribute until it reaches the start, thereby creating the complete path.



In [ ]:
class NODE:
    def __init__(self, position, g=float('inf'), h=0, parent=None):
        self.position = position
        self.g = g
        self.h = h
        self.f = g + h
        self.parent = parent

    # Nodes will be compared based on their f value, we use this because of the heapq
    def __lt__(self, other):
        return self.f < other.f

    # Function to represent the node
    def __repr__(self):
        return f"Node(position={self.position}, g={self.g}, h={self.h}, f={self.f})"

    @staticmethod
    def reconstruct_path(node):
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

### Heuristic Function: Octile Distance

The heuristic function used in this implementation is based on the **Octile distance**, which is suitable when diagonal movement is allowed (resulting in 8 possible movement directions). This heuristic ensures an efficient estimation of the shortest path by considering both orthogonal and diagonal moves.
The values assigned to these movements are:
- Vertical/Horizontal Movement Cost = 1
- Diagonal Movement Cost = $\sqrt{2} \approx 1.414$


In [ ]:
# Heuristic function using Octile distance (allowing 8 directions of movement)
def Heuristic_function(node, goal):
    dx = abs(node.position[0] - goal.position[0])
    dy = abs(node.position[1] - goal.position[1])
    return max(dx, dy) + (math.sqrt(2) - 1) * min(dx, dy)

### A* Pathfinding Algorithm

The `A_star` function is the core of our path planning, implementing the A* search algorithm to find an optimal path from a start to a goal position on a grid-based map. Below, we provide a breakdown of how the algorithm operates:

#### Explanation of the Key Steps

- **Initialization**:
  - **Start and Goal Nodes**: The function begins by creating a start node (`START_NODE`) with `g = 0` (the cost to reach the start node is zero) and a goal node (`GOAL_NODE`).
  - **Heuristic Calculation**: The heuristic (`h`) for the start node is computed using the `Heuristic_function`.
  - **Data Structures**:
    - **Open List** (`OPEN_list`): A priority queue that stores nodes yet to be explored, sorted based on their `f` value (total estimated cost).
    - **Closed Set** (`CLOSED_set`): A set that tracks nodes that have already been explored.
    - **Nodes Dictionary** (`nodes`): Tracks nodes that have already been visited, ensuring that if a node is revisited, the algorithm always uses the shortest known path to reach it rather than simply using the most recent path.

- **Environment Matrix Representation**:
  - The **environment map** is represented as a matrix where:
    - **`0`** indicates a **free path** that the algorithm can traverse.
    - **`1`** indicates an **obstacle** that cannot be crossed.

- **Search Loop**:
  - The algorithm proceeds until the **open list** is empty. It repeatedly pops the node with the **lowest `f` value** from the open list.
  - If the **current node** is already in the closed set, it is skipped to avoid redundant processing. Otherwise, it is added to the closed set for tracking purposes.

- **Goal Check**:
  - If the current node's position matches the goal, the function reconstructs the **optimal path** using the `reconstruct_path()` method.

- **Exploration of Neighboring Nodes**:
  - **Moves**: The possible moves include 8 directions — vertical, horizontal, and diagonal.
  - For each potential move:
    - The **neighbor's position** is calculated, and checks are performed to ensure that it is within bounds and is not an obstacle.
    - If the position is already in the closed set, it is skipped.
    - The **movement cost** is calculated:
      - **Vertical/Horizontal Movement Cost** = 1
      - **Diagonal Movement Cost** = $\sqrt{2} \approx 1.414$
  - The **tentative cost (`g_tentative`)** to reach the neighbor is computed by adding the movement cost to the `g` value of the current node.

- **Neighbor Node Handling**:
  - If the neighbor node does not exist in the `nodes` dictionary, a **new node** is created.
  - If the newly found path to the neighbor is **more efficient** than any previously known path (i.e., has a lower `g` value), the neighbor node's values are updated to reflect this improvement.
  - The neighbor node is then added to the **open list** for future exploration.

- **No Path Found**:
  - If the **open list** is exhausted without reaching the goal, the function returns **`None`** and prints "No path found."

In [ ]:
def A_star(environment_map, start_position, goal_position):

    # Create the start and goal nodes
    start_node = NODE(start_position, g=0)
    goal_node = NODE(goal_position)

    # Set the heuristic value for the start node
    start_node.h = Heuristic_function(start_node, goal_node)
    start_node.f = start_node.g + start_node.h

    # Initialize the open and closed lists
    open_list = []
    heapq.heappush(open_list, start_node)
    closed_set = set()

    # Dictionary to keep track of nodes
    nodes = {}
    nodes[tuple(start_node.position)] = start_node

    while open_list:
        # Pop the node with the lowest f value
        current_node = heapq.heappop(open_list)

        # If the current node is in the closed set, skip it
        if tuple(current_node.position) in closed_set:
            continue

        # Add the current node's position to the closed set
        closed_set.add(tuple(current_node.position))

        # If the current node is the goal, reconstruct the path
        if current_node.position == goal_node.position:
            path = NODE.reconstruct_path(current_node)
            return path

        # Possible moves: 8 directions (including diagonals)
        moves = [[1, 0], [0, 1], [-1, 0], [0, -1],
                 [1, 1], [-1, -1], [1, -1], [-1, 1]]

        # Explore neighbors
        for move in moves:
            neighbor_position = [current_node.position[0] + move[0],
                                 current_node.position[1] + move[1]]
            neighbor_pos = tuple(neighbor_position)

            # Skip if out of bounds or obstacle
            if (neighbor_position[0] < 0 or neighbor_position[0] >= environment_map.shape[0] or
                neighbor_position[1] < 0 or neighbor_position[1] >= environment_map.shape[1] or
                environment_map[neighbor_position[0], neighbor_position[1]] == 1):
                continue

            # Skip if in closed set
            if neighbor_pos in closed_set:
                continue

            # Calculate movement cost (diagonal movement cost is sqrt(2))
            dx = abs(move[0])
            dy = abs(move[1])
            movement_cost = math.sqrt(2) if dx == 1 and dy == 1 else 1

            # Calculate tentative g value
            g_tentative = current_node.g + movement_cost

            # Create or get the neighbor node
            if neighbor_pos not in nodes:
                neighbor_node = NODE(neighbor_position)
                nodes[neighbor_pos] = neighbor_node
            else:
                neighbor_node = nodes[neighbor_pos]

            # If this path to neighbor is better, record it
            if g_tentative < neighbor_node.g:
                neighbor_node.g = g_tentative
                neighbor_node.h = Heuristic_function(neighbor_node, goal_node)
                neighbor_node.f = neighbor_node.g + neighbor_node.h
                neighbor_node.parent = current_node

                # Add the neighbor to the open list
                heapq.heappush(open_list, neighbor_node)

    print("No path found.")
    return None


### Example: Finding the Treasure

To illustrate how our A* pathfinding algorithm works, we present a simple example: the goal is to reach the **treasure** in the shortest possible path.

In the following grid map:

- The **starting point** is marked by a **pirate boat**, and the **goal** is to reach the **treasure**.
- The algorithm efficiently navigates through free paths while avoiding obstacles to determine the optimal route.

Below is an illustration of the grid:
<div style="text-align: center;">
    <img src="images/maze_map_example.png" alt="Maze map" style="width:40%; height:auto;">
</div>

In [ ]:
def convert_image_to_binary_array(image_path):
    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    image = image.convert("L")

    # Set the binary threshold
    binary_threshold = 200

    # Convert grayscale to binary image
    bw_image = image.point(lambda p: p < binary_threshold and 1)

    # Convert image to numpy array
    binary_matrix = np.array(bw_image)
    return binary_matrix

def swap_path_coordinates(path_coordinates):
    # Swap the x and y coordinates of the path (to match the image)
    swapped_path = [[y, x] for x, y in path_coordinates]
    return swapped_path

def display_path(maze_name, path):
    # Load the maze image
    maze_path = maze_name
    maze_img = Image.open(maze_path)

    # Convert the maze image to RGB (to draw in color)
    maze_img = maze_img.convert("RGB")

    # Swap the x and y coordinates of the path (to match the image)
    path = swap_path_coordinates(path)

    # Create a drawing object
    draw = ImageDraw.Draw(maze_img)

    # Draw the path (red line or points)
    for i in range(len(path) - 1):
        x1, y1 = path[i]
        x2, y2 = path[i + 1]
        draw.line((x1, y1, x2, y2), fill=(255, 0, 0), width=4)  # Red line with width 2

    # Save and show the image
    maze_img.save("maze_with_path.png")
    maze_img.show()
    return

# Test the A* algorithm

# Define the start and goal positions
start_position = [340, 120]
goal_position = [310, 793]

# Load the binary matrix from the image
binary_matrix = convert_image_to_binary_array("maze_map_example.png")

# Find the path using A*
path = A_star(binary_matrix, start_position, goal_position)

# Display the path on the maze image
display_path("maze_map_example.png", path)

### Result

The resulting path is shown below, marked in **red** to indicate the shortest route taken by the pirate boat to reach the treasure.
<div style="text-align: center;">
    <img src="images/maze_with_path.png" alt="maze with path" style="width:40%; height:auto;">
</div>

## Local Navigation

### Local Navigation

We initially tried to implement a wall-following technique for obstacle avoidance, but due to the nature of our map—specifically the narrow paths the Thymio has to navigate—we opted for a different approach that suited our environment better. To handle these situations more effectively, we decided to make our Thymio “smarter” by leveraging the **global map** we defined earlier. 

When computing the global map, we generate a **distance map**. This distance map converts our environment into a grid of distances, where each cell represents how far it is from the nearest wall or object in the map. Essentially, it gives the Thymio a better sense of the available space around it in a precomputed, global way, rather than relying solely on its proximity sensors. This makes navigation in tight spaces much easier and more efficient.

When the Thymio detects a local 3D obstacle (using its proximity sensors), it enters the **avoidance state ( state 1)**. In this state, the decision on how to avoid the obstacle is based on determining which side (left or right) has more free space. This decision is made by the `check_sides` function, which uses the distance map to evaluate the distance to obstacles on either side of the robot. Specifically, it projects left and right from the robot’s current position, checking where there is more space to turn. This is a simple yet effective way to choose the best side to maneuver toward.

After determining which side has more free space, the Thymio executes a simple maneuver in the `avoid_obstacle` function: it turns toward the side with more space, reorients itself, and then moves forward to get back on track. The duration and execution of this maneuver vary depending on which proximity sensors are triggered ,their thresholds and the turning decision—larger turns are performed when the front sensors detect an obstacle. The timer values for these maneuvers were determined through empirical experimentation. Once the obstacle is cleared, the Thymio goes to **state 2** recalculates its path to the goal using our A* algorithm, ensuring it continues its journey efficiently without getting stuck.

This approach allowed us to accomplish the local navigation task in a way that suited our application.

## Filtering

The Thymio's position is estimated using a Kalman Filter, specifically an Exended Kalman filter due to the non-linearity of the differential-drive kinematics. Another possible option was a Particle Filter, however, it is less computationally efficient. Given the need for real-time estimation, the EKF was a better choice. 

The observations are given by two sources : the odometry and the camera measurements. The noise of those two sources are assumed to be Gaussian which aligns with the assumptions of the EKF. 

The code of the Kalman filter can be found in *Kalman.py*.

### Parameters 
#### Input parameters

| Input parameters | Descriptions | Units |
|------------------|-----------------|-----------------|
| `state_est_prev`   | Previous state estimate = [x_prev, y_prev, yaw_prev, v_prev, omega_prev]    | [mm, mm, rad, mm/s, rad/s]   |
| `control_vect_prev`     | Previous control vector = [v_input_prev, omega_input_prev] |  [mm/s, rad/s]  |
| `P_prev` | Previous state a posteriori covariance | [mm $^2$/s $^2$] |
| `obs_camera`   | Observation vector from the camera = [x_cam, y_cam, yaw_cam]    | [mm, mm, rad]   |
| `obs_odometry`   | Observation vector from the odometry = [v_odo, omega_odo]    | [mm/s, rad/s]   |
| `camera_state`   | State of the camera = True if available, False otherwise    | [Bool]   |
| `Ts`   | Time step   | [s]  |

#### Output parameters
| Output parameters | Descriptions | Units |
|------------------|-----------------|-----------------|
| `state_est`   | Current state estimate = [x_est, y_est, yaw_est, v_est, omega_est]    | [mm, mm, rad, mm/s, rad/s]   |
| `P_est` | Current state a posteriori covariance | [mm $^2$/s $^2$] |




### Space state estimation 
The EKF was implemented with the help of :
- "Lesson 7/8- Uncertainties" from the course Basics of Mobile Robotics (MICRO-452) given by Francesco Mondada.
- Automatic Addison https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
- Wikipedia page of the Extended Kalman Filter : https://en.wikipedia.org/wiki/Extended_Kalman_filter#Discrete-time_predict_and_update_equations


In the EKF, the current state is estimated from the previous one using the dynamic of the model: 

$$
state_{k} = \begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix} = \begin{bmatrix} 
x_{k-1} + \cos(\gamma_{k-1}) \cdot T_s \cdot v_{k-1}\\
y_{k-1} + \sin(\gamma_{k-1}) \cdot T_s \cdot v_{k-1}\\
\gamma_{k-1} + Ts \cdot \omega_{k-1}\\
v_{input}\\
\omega_{input}
\end{bmatrix} 
$$

Here, $v_{input}$ and $\omega_{input}$ are the command given to the Thymio.


### Predicted covariance 
The predicted covariance is given by :
$$
P_{pred} =  G\cdot P_{k-1}\cdot G^T + Q
$$

The matrix $G$ is given by the Jacobian the motion model :

$$
G = \begin{bmatrix}
1 & 0 & -\sin(\gamma_{k-1}) \cdot T_s \cdot v_{k-1} & \cos(\gamma_{k-1}) \cdot T_s & 0 \\
0 & 1 & \cos(\gamma_{k-1}) \cdot T_s \cdot v_{k-1} & \sin(\gamma_{k-1}) \cdot T_s & 0 \\
0 & 0 & 1 & 0 & T_s \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}
$$



### Covariance matrices
**State covariance** :
$ Q = \begin{bmatrix}
q_x & 0 & 0 & 0 & 0 \\
0 & q_y & 0 & 0 & 0 \\
0 & 0 & q_{\gamma} & 0 & 0 \\
0 & 0 & 0 & q_v & 0 \\
0 & 0 & 0 & 0 & q_{\omega}
\end{bmatrix}$

**Measurement covariance** : 
- Camera available : $R = \begin{bmatrix}
r_x & 0 & 0 & 0 & 0 \\
0 & r_y & 0 & 0 & 0 \\
0 & 0 & r_{\gamma} & 0 & 0 \\
0 & 0 & 0 & r_v & 0 \\
0 & 0 & 0 & 0 & r_{\omega}
\end{bmatrix}$

- Camera not available : 
When the camera in not available, the covariance of the measured positions and orientation are infinite. 

    Hence :
$R_{\text{no\_camera}} = \begin{bmatrix}
∞ & 0 & 0 & 0 & 0 \\
0 & ∞ & 0 & 0 & 0 \\
0 & 0 & ∞ & 0 & 0 \\
0 & 0 & 0 & r_v & 0 \\
0 & 0 & 0 & 0 & r_{\omega}
\end{bmatrix}$

**Choice of the parameters:**

We determined the parameters via a little experiment detailled in the file covariance_estimation.ipynb. This experiment is based on "Solutions for exercise session 8" from the course Basics of Mobile Robotics (MICRO-452) given by Francesco Mondada.

First, we gave the instruction to the Thymio to go straight to a certain speed (50 thymiounit for both wheel) and measured the actual speed of the wheels.

The avergage speed is calculated as : $avg\_ speed =\frac{v_{left}+v_{right}}{2}$

<div style="text-align: center;">
    <img src="images/speeds_for_covariance.png" alt="Alt Text" style="width: 50%;"/>
    <img src="images/Image_conversion.png" alt="Alt Text" style="width: 50%;"/>
</div>

The measured distance that the robot did is: d=280mm.

Hence, we can calculate a conversion factor from Thymio-speed to mm/s : 
$$
thymio\_ speed\_ to\_ mms= \frac{d}{T_s*nb_{timeSteps}*thymio\_ speed}=0.4375$$

Were $T_s$ is the time between 2 measures.

**Covariance of v and w**

To find the covariance of v and w we made 3 experciences. To have the most accurate results as possible, we did 3 times each experiments. The following results use the assumption that that half of the variance is caused by the perturbations and the other half by the measurement :


**Experience 1 :** the thymio goes forward (speed (50,50))

>Speed covariances found = 4.21 , 8.14 and 5.75 $[mm^2/s^2]$ 

>Angular speed covariances found = 0.0014, 0.0032 and 0.0024  $[mm^2/s^2]$ 

**Expercience 2 :** the thymio turn on itself (speed (50,0) as the speed of the wheels won't be negatives in our project)

>Speed covariances found = 5.62, 5.81, 8.43

>Angular speed covariances found = 0.0027, 0.0025, 0.0038

**Expercience 3 :** the thymio follows a curved trajectory (speed (50,40))

>Speed covariances found = 0.59, 0.58, 0.85

>Angular speed covariances found = 0.0002, 0.0002, 0.0004

In the table below one can find the means of the results 

| | Experience 1 | Experience 2 | Experience 3  |
|-----------------|------------------|-----------------|-----------------|
|$q_v=v_v [mm^2/s^2]$| 6.03 |  6.62  |  0.67  |
|$q_w = r_w [rad^2/s^2]$|0.0023| 0.009| 0.00026 |

From this table, we decided to used the following covariances : 

$q_v = r_v = 6 $ $mm^2/s^2$

$ q_w = r_w = 0.009 $ $ rad^2/s^2$


**Sate covariances of x, y and $\gamma$**
The covariances of x and y are the same. We chose the same arbitrary value as the one in the Exercise 8 :

$q_x=q_y=q_p=0.04$

**Measurement covariances of x, y and $\gamma$**

We arbitrarily chose the values $r_x = r_y = q_p = 0.001$. We selected small values because we want the filter to place significant trust in the position and angle measurements provided by the camera, as they are the most accurate measurements we can obtain for the robot.


### Final covariance matrices
$$
Q = \begin{bmatrix}  
0.04 & 0 & 0 & 0 & 0 \\  
0 & 0.04 & 0 & 0 & 0 \\  
0 & 0 & 0.01 & 0 & 0 \\  
0 & 0 & 0 & 6 & 0 \\  
0 & 0 & 0 & 0 & 0.009  
\end{bmatrix}, R = \begin{bmatrix}  
0.001 & 0 & 0 & 0 & 0 \\  
0 & 0.001 & 0 & 0 & 0 \\  
0 & 0 & 0.001 & 0 & 0 \\  
0 & 0 & 0 & 6 & 0 \\  
0 & 0 & 0 & 0 & 0.009  
\end{bmatrix}, R_{\text{no\_camera}} = \begin{bmatrix}  
\infty & 0 & 0 & 0 & 0 \\  
0 & \infty & 0 & 0 & 0 \\  
0 & 0 & \infty & 0 & 0 \\  
0 & 0 & 0 & 6 & 0 \\  
0 & 0 & 0 & 0 & 0.009  
\end{bmatrix} 
$$



### Observation/measurement model 

The observation model is given by : 
$$
y_k = H \cdot x_k 
$$
$$
\begin{bmatrix} 
y_1^k \\
y_2^k \\
y_3^k \\
y_4^k \\
y_5^k
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix} \cdot \begin{bmatrix} 
x_{k} \\
y_{k} \\
\gamma_{k} \\
v_{k}\\
\omega_{k}
\end{bmatrix}
$$

The observations are given by the camera for $x_{k}$, $y_{k}$ and $\gamma_{k}$ and by the odometry for $v_{k}$ and $\omega_{k}$.

# Motion Control Module

# Motion Control Module

Once the vision module is implemented to send the robot's position and angle, and the optimal path is computed, the motion control module comes into play to guide the robot toward its goal efficiently and reliably. Our A* algorithm computes a highly precise path, which we segment into closely spaced waypoints to ensure smooth and accurate motion. To navigate these waypoints, we implemented a simplified version of the Astolfi controller, which is ideal for the Thymio's non-holonomic differential drive system. Due to the short distances between waypoints, we implemented a more simplified version of the Astolfi controller then the one seen in the course, omitting the beta angle correction. The controller focuses on minimizing two critical variables: the distance to the goal ($\rho$) and the alignment error ($\alpha$), ensuring the robot aligns itself with its path and reaches each waypoint effectively.

![Controller Diagram](/controller.jpg)

The distance to the goal, $\rho$, is calculated as:

$$
\rho = \sqrt{(x_{goal} - x_{robot})^2 + (y_{goal} - y_{robot})^2}
$$

This represents the Euclidean distance between the robot's position and the target waypoint. 

The alignment error, $\alpha$, is defined as:

$$
\alpha = \arctan2(dy, dx) - \theta
$$

Here, $dx = x_{goal} - x_{robot}$ and $dy = y_{goal} - y_{robot}$, while $\theta$ is the robot's current orientation. The term $\arctan2(dy, dx)$ gives the direction to the waypoint relative to the robot's current position, and subtracting $\theta$ calculates the angular difference. To adapt to the inverted y-axis of the camera’s coordinate frame, we flipped the signs of the y-coordinates in our implementation, ensuring consistency in how the controller interprets these values.

The controller generates two outputs: linear velocity ($v$) and angular velocity ($\omega$), which are computed using proportional control laws. These are defined as:

$$
v = K_\rho \cdot \rho
$$

$$
\omega = K_\alpha \cdot \alpha
$$

Here, $K_\rho > 0$ and $K_\alpha > 0$ are gain parameters tuned for stability. The gain $K_\alpha$ is chosen to be greater than $K_\rho$ ($K_\alpha > K_\rho$) to ensure that the robot prioritizes alignment before moving forward, preventing overshooting or instability. Both $K_\rho$ and $K_\alpha$ must be positive to ensure the robot moves toward the goal while aligning itself correctly, maintaining stability by progressively reducing position and angular errors, as guaranteed by Lyapunov’s method.

To determine these gains, we used a simple method for initial parameter estimation. Starting with a straightforward scenario where the robot moves in a straight line, such as from (0,0,0) to (x,0), we defined a desired wheel speed for this motion and estimated the corresponding $v$, $\rho$ and estimated distance traveled. From this, we had an idea of starting values for $K_\rho$ to produce the required linear velocity. For $K_\alpha$, we chose a value slightly larger than $K_\rho$ to ensure the robot prioritizes angular alignment and stability reason. These initial guesses were then refined through empirical tuning, where we tested different scenarios, like turning or navigating along curved paths, and adjusted the parameters until the robot exhibited stable and smooth behavior.This iterative approach proved practical and effective for our implementation.

These velocities are then mapped to the Thymio's motor speeds in `compute_motor_speeds` function using the differential drive equations:

$$
v_{right} = v + \omega \cdot d
$$

$$
v_{left} = v - \omega \cdot d
$$

`compute_motor_speeds` also makes sure the velocities don't exceed a certain max_speed we precised, and converts them to thymio speed units.

Here, $d$ represents the distance between the robot's wheels. This mapping ensures that the Thymio can translate the computed velocities into physical movement.

A certain threshold is defined for the distance to the goal ($\rho$), which determines when the Thymio has successfully reached its target waypoint. By continuously recalculating the distance and alignment errors as the robot moves, the controller dynamically adjusts the velocities, enabling the Thymio to navigate its path with accuracy and stability. This method combines the precision of proportional control with the adaptability required for real-world navigation.

All relavant functions to this part are in the `Control.py` module.

## Main

**Decision Map :**
<div style="text-align: center;">
    <img src="images/decision_map.jpg" alt="Alt Text" style="width: 50%;"/>
</div>

Imports from each module:

In [ ]:
import Modules.Control as Control
import matplotlib.pyplot as plt
import Modules.Path_planning as Path_planning
import tdmclient.notebook
import Modules.ThymioCommands as ThymioCommands
import Modules.LocalNav as LocalNav
import time
import cv2
import numpy as np
import Modules.Kalman as Kalman
from Modules.ThymioVision import ThymioVision # Computer vision package
from scipy.ndimage import distance_transform_edt

In [ ]:
import tdmclient.notebook
import time
#fonctions de Julie 
@tdmclient.notebook.sync_var 
def set_speed(right_speed,left_speed):
    global motor_right_target, motor_left_target
    motor_right_target=right_speed
    motor_left_target=left_speed

@tdmclient.notebook.sync_var
def stop_thymio():
    global motor_right_target,motor_left_target
    motor_right_target=0
    motor_left_target=0

@tdmclient.notebook.sync_var
def get_speed():
    global motor_right_speed, motor_left_speed
    return motor_right_speed,motor_left_speed

def speed_convesion(r_speed,l_speed):
    thymio_speed_to_mms = 0.4375 # value found in covariance_estimation

    #odometry 
    avg_thymio_speed = (r_speed + l_speed) / 2
    speed = avg_thymio_speed * thymio_speed_to_mms # [mm/s]
    return speed

def angular_vel_conversion(r_speed,l_speed):
    d = 95 # distance between the 2 wheels [mm]
    thymio_speed_to_mms = 0.4375 # value found in covariance_estimation
    
    difference_speed = l_speed - r_speed
    omega = difference_speed * thymio_speed_to_mms / d # [rad/s]

    return omega

@tdmclient.notebook.sync_var
def get_proximity_values():
    global prox_horizontal
    prox = prox_horizontal
    prox_front = prox[2]
    prox_left = prox[0]
    prox_left_front = prox[1]
    prox_right_front = prox[3]
    prox_right = prox[4]
    return prox_front, prox_left, prox_left_front, prox_right_front, prox_right

def avoid_obstacle(vc, duration = 2, decision="right"):
    """
    Perform a simplified L-shaped maneuver for obstacle avoidance using time.sleep.
    Args:
        decision (str): Turning direction ("right" or "left").
    """
    print("Starting obstacle avoidance maneuver...")

    # Phase 1: Turn away from the obstacle
    print(f"Phase 1: Turning {decision} away from the obstacle.")
    curr_time = time.time()
    if decision == "right":
        set_speed(-100, 100)  # Turn right
    elif decision == "left":
        set_speed(100, -100)  # Turn left
    while time.time() - curr_time < duration:
        ret,frame = vc.read()
        if ret:
            cv2.imshow("Camera view", frame)
        key = cv2.waitKey(50)
    
    # Phase 2: Move forward to clear the obstacle
    curr_time = time.time()
    print("Phase 2: Moving forward to clear the obstacle.")
    set_speed(100, 100)  
    while time.time() - curr_time < duration+1:
        ret,frame = vc.read()
        if ret:
            cv2.imshow("Camera view", frame)
        key = cv2.waitKey(50)

    # Phase 3: Turn back to original direction
    curr_time = time.time()
    print(f"Phase 3: Returning to original direction (opposite of {decision}).")
    if decision == "right":
        set_speed(100, -100)  # Turn left
    elif decision == "left":
        set_speed(-100, 100)  # Turn right
    while time.time() - curr_time < duration:
        ret,frame = vc.read()
        if ret:
            cv2.imshow("Camera view", frame)
        cv2.waitKey(50)

    # Stop the robot
    stop_thymio()
    print("Obstacle avoidance completed.")


Begin the thymio, calibrate for demo

In [ ]:
await tdmclient.notebook.start()

In [ ]:
global leds_circle
leds_circle=[32,32,0,0,0,0,0,32] # Turn on thymio lights for tracking

In [ ]:
await tdmclient.notebook.stop()

In [ ]:
ThymioVision.calibrateCameraPos(0) #Calibrate the corners to the map.
frame = ThymioVision.captureImageOnCommand(0) # Capture the image to define the map
edges = ThymioVision.getEdges(frame, verbose=True)
start = ThymioVision.getThymioPose(frame, verbose=True)
end = ThymioVision.detectGoal(frame, verbose=True)

In [ ]:
map = ThymioVision.getMap(frame, verbose=True)

In [ ]:
# Define the inputs for the path_planning module
obstacles = map[0]
start = map[1]
end = map[2]
# adjusting x/y to row/col -> flip them.
correctedStart = [start[1], start[0]]
correctedGoal = [end[1], end[0]]

# Determine best path from map
path = Path_planning.A_star(obstacles, correctedStart, correctedGoal)

# Break the path into waypoints that the Thymio can follow.
# Swap back to pixel coordinates
swapped = Path_planning.swap_path_coordinates(path)
waypoints = []
for point in swapped:
    x, y = ThymioVision.pixelToRealSpace(point)
    waypoints.append((x*10, y*10)) #convert to mm!
waypoints = Control.segment_path(waypoints, step=40)

lx = []
ly = []
for wp in path:
    lx.append(wp[0])
    ly.append(wp[1])

# Display path on frame
plt.imshow(frame)
plt.plot(ly, lx)
plt.show()

In [ ]:
dist_map = distance_transform_edt(map[0] == 0)

flipped_dist_map = np.flipud(dist_map)

# Plot the distance transform
plt.figure(figsize=(10, 10))
plt.imshow(dist_map.T, origin='lower', cmap='viridis', extent=(0, 100, 0, 100))
plt.colorbar(label="Distance to Nearest Wall")
plt.title("Distance Transform of the Map")
plt.xlabel("X")
plt.ylabel("Y")
plt.grid(True)
plt.show()

Main Movement Loop!

In [ ]:
import cv2

# BEGIN CAMERA CAPTURE
cv2.namedWindow("Camera view")
vc=cv2.VideoCapture(0)
print("vc captured")

###################### INITIALIZE STATE FROM MAP READ ######################

obstSpeedGain = [2, 1, -1, -1.5, -2.5]  # Gains for obstacle avoidance
OBSTACLE_THRESH_LOW_FRONT = 8 # Threshold to stop avoiding
OBSTACLE_THRESH_HIGH_FRONT = 30 # Threshold to start avoiding
OBSTACLE_THRESH_HIGH_SIDE = 40 # Threshold to start avoiding
thymio_state = 0  # 0 = Goal tracking, 1 = Obstacle avoidance, 2 = kidnapping
goal_tolerance = 30


state_est=[0,0,0,0,0] 
ret,frame=vc.read()
if ret:
    pose = ThymioVision.getThymioPose(frame) #pixel location of thymio
    obs_camera = ThymioVision.pixelToRealSpace(pose[0:2]) #returns cm, *10 to get mm
    obs_camera = (obs_camera[0]*10, obs_camera[1]*10, pose[2]) #thymio position in mm to give to kalman filter
    state_est=[obs_camera[0],obs_camera[1],obs_camera[2],0,0]   
P_est=np.diag([0.1,0.1,0.1,0.1,0.1])


###################### LOGGING DATA ######################
trajectory = []  # To store [x, y] positions
trajectory.append(obs_camera[0:2])
camera_trajectory = [] #trajectory mapped by camera
trajectory = []  # Logs (x, y) positions
metrics_rho = []  # Logs for distance to goal
metrics_alpha = []  # Logs for angle to goal
time_prev = time.time()

###################### BEGIN MOTION LOOP ######################
while len(waypoints) > 0: # while we have goals in life
    actual_time = time.time()
    Ts = actual_time - time_prev
    if Ts < 0.15:
        time.sleep(0.15-Ts)
    time_prev = actual_time

    waypoint = waypoints[0] # current goal

    ################### SENSOR READS ###################
    ret,frame=vc.read()
    if ret:
        pose = ThymioVision.getThymioPose(frame)#pixel location of thymio
        if pose[0] is not None:
            obs_camera = ThymioVision.pixelToRealSpace(pose[0:2]) #returns cm, *10 to get mm
            obs_camera = (obs_camera[0]*10, obs_camera[1]*10, pose[2]) #thymio position in mm to give to kalman filter
            camera_trajectory.append(obs_camera) #position in mm appended to log
            arrowStart = pose[0:2]
            arrowEnd = (int(pose[0] + 50 * np.cos(pose[2])), int(pose[1] - 50 * np.sin(pose[2])))
            cv2.arrowedLine(frame, arrowStart, arrowEnd, color=(0,0,255), thickness=3) #camera pos
            for i in range(len(swapped)-1):
                cv2.line(frame, swapped[i], swapped[i+1], color=(255, 0 , 50), thickness=2)
            # convert trajectory onto map
        if len(trajectory) > 0:
            lastPos = trajectory[-1]
            lastPos = (lastPos[0]/10, lastPos[1]/10)
            pixelPred = ThymioVision.realSpaceToPixel(lastPos)
            cv2.circle(frame, (pixelPred), 3, (0,255,0),-1) #est pos
        cv2.imshow("Camera view", frame)
        key = cv2.waitKey(50)
    if key == ord('q'):
        break
    if not ret or pose[0] is None:
        camera_state=False
    else:
        camera_state=True
    print(camera_state)

    prox_front, prox_left, prox_left_front, prox_right_front, prox_right = get_proximity_values()
    proximity_values = [prox_left, prox_left_front, prox_front, prox_right_front, prox_right]

    ###################### CHECK THYMIO STATE ######################

    # Check for obstacle state transitions
    if thymio_state == 0:
        # Switch to obstacle avoidance if an obstacle is detected
        if any((prox // 100) > OBSTACLE_THRESH_HIGH_FRONT for prox in proximity_values[1:4]) or prox_right//100 > OBSTACLE_THRESH_HIGH_FRONT or prox_left//100 > OBSTACLE_THRESH_HIGH_FRONT:
            thymio_state = 1
        elif len(trajectory) >= 1 and ((obs_camera[0]-trajectory[-1][0])**2 + (obs_camera[1]-trajectory[-1][1])**2) > 500:
            print(obs_camera)
            print(trajectory[-1])
            # if change between states is too rapid, we have been kidnapped. recalculate and attempt again
            thymio_state = 2
            continue
    elif thymio_state == 1:
    # Switch back to goal tracking if obstacles are cleared
        if all((prox // 100) < OBSTACLE_THRESH_LOW_FRONT for prox in proximity_values):
            print("Obstacle avoided")
            #set_speed(20, 20)  #Continue getting away from obstacle for a short time ?   
            #stop_thymio() #idk if i keep it 
            thymio_state = 2 # set state to "kidnapped." Will recalculate the path and continue.

    ###################### GLOBAL NAVIGATION ######################
    if thymio_state == 0:
        print("Global Nav")
        # input_left_speed, input_right_speed, reached , v_f , omega_f  = Control.move_to_waypoint(state_est, waypoint, goal_tolerance)
        # input_left_speed, input_right_speed, reached , v_f , omega_f  = Control.move_to_waypoint(obs_camera, waypoint, goal_tolerance)
        if not camera_state:
            input_left_speed, input_right_speed, reached , v_f , omega_f  = Control.move_to_waypoint(state_est, waypoint, goal_tolerance*2)
            set_speed(int(input_right_speed/2), int(input_left_speed/2))
        else:
            input_left_speed, input_right_speed, reached , v_f , omega_f  = Control.move_to_waypoint(state_est, waypoint, goal_tolerance)
            set_speed(input_right_speed, input_left_speed)
    ###################### LOCAL NAVIGATION ######################   
    elif thymio_state == 1:
        print("Avoidance")
        decision, left_dist, right_dist = LocalNav.check_sides(state_est[0:3], flipped_dist_map)
        print(f"Turning decision: {decision}, Left Distance: {left_dist}, Right Distance: {right_dist}")
        if not(prox_right//100 > OBSTACLE_THRESH_HIGH_SIDE) and not (prox_left//100 > OBSTACLE_THRESH_HIGH_SIDE):
            avoid_obstacle(vc=vc, duration=2, decision=decision)
        else:
            avoid_obstacle(vc=vc, duration=1, decision=decision)

    ###################### KIDNAPPING RECALCULATION ###################### 
    else:
        print("Kidnapped") ##TODO FIX NO PATH FOUND ERROR
        # stop the thymio
        input_left_speed, input_right_speed, reached , v_f , omega_f = 0, 0, False, 0, 0
        set_speed(input_right_speed, input_left_speed)
        #if the camera is reading, update the path
        if camera_state:
            # convert trajectory onto map
            if len(trajectory) > 0:
                lastPos = trajectory[-1]
                lastPos = (lastPos[0]/10, lastPos[1]/10)
                pixelPred = ThymioVision.realSpaceToPixel(lastPos)
                cv2.circle(frame, (pixelPred), 3, (0,255,0),-1) #est pos
            cv2.imshow("Camera view", frame)
            path = None
            while not path:
                ret,frame=vc.read()
                if ret:
                    pose = ThymioVision.getThymioPose(frame)#pixel location of thymio
                    if pose[0] is not None:
                        path = Path_planning.A_star(obstacles, (pose[1],pose[0]), correctedGoal)
                    cv2.imshow("Camera view", frame)
            swapped = Path_planning.swap_path_coordinates(path)
            waypoints = []
            for point in swapped:
                x, y = ThymioVision.pixelToRealSpace(point)
                waypoints.append((x*10, y*10)) #convert to mm!
            waypoints = Control.segment_path(waypoints, step=40)
            trajectory.append(obs_camera[0:2]) #append most recent location
            thymio_state = 0 # go back to global nav

        
    ################### UPDATE STATE ESTIMATE ###################

    # Control input
    input_v = v_f #Ca jsp si je remplace par actual_v et actual_w
    input_w = omega_f
    control_vect_prev = [input_v, input_w]
    
    r_speed_odo, l_speed_odo = get_speed()
    actual_v = speed_convesion(r_speed_odo, l_speed_odo)
    actual_w = angular_vel_conversion(r_speed_odo, l_speed_odo)
    obs_odometry = [actual_v, actual_w]

    # Update state with Kalman filter
    state_est, P_est = Kalman.ekf(state_est, control_vect_prev, P_est, obs_camera, obs_odometry, camera_state,Ts)
    state_est[2]=Control.normalize_angle(state_est[2])
    # Log trajectory
    trajectory.append((state_est[0], state_est[1]))
    
    # Calculate and log metrics
    rho = np.sqrt((waypoint[0] - state_est[0])**2 + (waypoint[1] - state_est[1])**2)
    alpha = Control.normalize_angle(np.arctan2(waypoint[1] - state_est[1], waypoint[0] - state_est[0]) - state_est[2])
    metrics_rho.append(rho)
    metrics_alpha.append(alpha)
    
    # Print updated state
    # print(ThymioVision.pixelToRealSpace(pose[0:2]))
    print(f"Position Actuelle camera  : x = {obs_camera[0]}, y = {obs_camera[1]}, theta = {obs_camera[2]}") 
    print(f"Position Actuelle odometry: x = {state_est[0]}, y = {state_est[1]}, theta = {state_est[2]}") 

    if reached:
        waypoints.pop(0)
        print(f"Reached waypoint: {waypoint}")
        # Stop the robot at the waypoint
        stop_thymio()

vc.release()
stop_thymio()
cv2.destroyAllWindows()
print("Trajectory completed!")


## Results

Video of our project results : https://drive.google.com/file/d/1tEe2iRlEP_2pRoXrDznQHucG7x7W3pVn/view?usp=sharing

## Sources 
- Course "Basics of Mobile Robotics" by Francesco Mondada
- ChatGPT

### Global Navigation
- *"Space Robotics Systems"*, delivered by Prof. Antonio Genova and Dr. Edoardo Del Vecchio from the Department of Mechanical and Aerospace Engineering, Sapienza University of Rome. 

### Filtering 
- "Lesson 7/8- Uncertainties" from the course Basics of Mobile Robotics (MICRO-452) given by Francesco Mondada.
- "Solutions for exercise session 8" from the course Basics of Mobile Robotics (MICRO-452) given by Francesco Mondada.
- Automatic Addison https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
- Wikipedia page of the Extended Kalman Filter : https://en.wikipedia.org/wiki/Extended_Kalman_filter#Discrete-time_predict_and_update_equations
